In [1]:
# Librerie
import os
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import random
sns.set_style('whitegrid')
import itertools
import json
import math

from math import factorial
from IPython.core.display import display, HTML

In [2]:
df = pd.read_csv('../../src/clean_csv/clean_dataframe.csv', sep='\t')
df.columns = ['index', 'id_par','label', 'statement', 'subject', 'speaker', 
              'job', 'state', 'party', 'context']
df = df.drop(['index'], axis=1)

df = df[~df['id_par'].isin(['1779.json'])]
# replace string
t_repl = {'barely-true': "BARELYTRUE",
          'false': "FALSE",
          'half-true': "HALFTRUE",
          'mostly-true': "MOSTLYTRUE",
          'pants-fire' : "LIE",
          'true' : "TRUE"
        }
party_repl = {'democrat' : "DEM",
              'republican' : "REP"
            }

def f1(l):
    return [ t_repl.get(item,item) for item in l ]
def f2(l):
    return [ party_repl.get(item,item) for item in l ]
df['label'] = df.label.str.split('<').apply(f1).str.join('')
df['party'] = df.party.str.split('<').apply(f2).str.join('')

df['name'] = '-1'
# Prendo solo Democratici, Repubblicani 
sub_df = df[(df.party == "DEM") | (df.party == "REP")]

display(sub_df.tail())
display(sub_df.shape)
display(df.head())
display(df.shape)

,id_par,label,statement,subject,speaker,job,state,party,context,name
12779,5671.json,FALSE,I think its seven or eight of the California s...,"education,history",rick-santorum,NaN,Pennsylvania,REP,in a speech in Wisconsin,-1
12781,7334.json,HALFTRUE,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,REP,a news conference,-1
12782,9788.json,BARELYTRUE,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,DEM,"on ABC's ""This Week""",-1
12784,3186.json,BARELYTRUE,Says an EPA permit languished under Strickland...,"environment,government-efficiency",john-kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,REP,a news conference,-1
12785,6743.json,FALSE,Says the governor is going around the state ta...,"state-budget,state-finances,taxes",john-burzichelli,NaN,NaN,DEM,an interview with NJToday,-1


(9798, 10)

,id_par,label,statement,subject,speaker,job,state,party,context,name
0,10540.json,HALFTRUE,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,DEM,a floor speech.,-1
1,324.json,MOSTLYTRUE,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,DEM,Denver,-1
2,1123.json,FALSE,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,a news release,-1
3,9028.json,HALFTRUE,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,DEM,an interview on CNN,-1
4,12465.json,TRUE,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,REP,a an online opinion-piece,-1


(12785, 10)

In [10]:
filtered = pd.DataFrame(columns = sub_df.columns)
categories = ["LIE", "FALSE", "BARELYTRUE", "HALFTRUE", "MOSTLYTRUE", "TRUE"]
par = ["DEM", "REP"]
docs_per_cat = 20
docs_per_cat_by_party = int(docs_per_cat/len(par))

for cat in categories:
    for p in par:
        c = sub_df['label'] == cat
        p = sub_df['party'] == p
        # Prendo i primi docs_per_cat per ogni label
        sub = sub_df[c & p]
        
        sub = sub.head(docs_per_cat_by_party)
        assert sub.shape[0] == docs_per_cat/2, 'Non ho abbastanza docs per tutte le categorie'
        # Nomino i documenti
        sub['name'] = [cat + '_doc' + str(x + 1) for x in range(len(sub))]
        # Concateno il tutto
        filtered  = pd.concat([filtered, sub])

filtered["name"] = filtered["party"].map(str) + '_' +  filtered["name"]
# Compongo il Json con i campi necessari
filtered["js"] = [ {"id_par": filtered.iloc[x]['id_par'],
                    "name": filtered.iloc[x]['name'],
                    "statement": (filtered.iloc[x]['statement']).replace("'", ""),
                    "speaker": filtered.iloc[x]['speaker'],
                    "job": filtered.iloc[x]['job'],
                    "context": filtered.iloc[x]['context'],
                    "party": filtered.iloc[x]['party'],
                    }
                  for x in range(len(filtered))]
display(filtered.groupby(['party']).count())
filtered = filtered[['name', 'js']]

# Aggiungo Gold High | Low
high = {'id_par':'HIGH',
        'name':'HIGH',
        'statement':'Yes, We can!',
        'speaker':'Barack Obama',
        'job':'Former U.S. President',
        'context':'Public appearance',
        'party':'democrat',
        }
low = {'id_par':'LOW',
        'name':'LOW',
        'statement':'if a were not a sumo wrestler, I would be a football player.',
        'speaker':'Abraham Lincoln',
        'job':'Former U.S. President',
        'context':'Public appearance',
        'party':'republican'
        }

filtered = filtered.append({'name' : 'HIGH', 'js': high}, ignore_index = True)
filtered = filtered.append({'name' : 'LOW', 'js': low}, ignore_index = True)

display(filtered.tail())

,id_par,label,statement,subject,speaker,job,state,context,name,js
party,,,,,,,,,,
DEM,60,60,60,60,60,48,57,60,60,60
REP,60,60,60,60,60,48,52,60,60,60


,name,js
117,REP_TRUE_doc8,"{'id_par': '3054.json', 'name': 'REP_TRUE_doc8..."
118,REP_TRUE_doc9,"{'id_par': '4681.json', 'name': 'REP_TRUE_doc9..."
119,REP_TRUE_doc10,"{'id_par': '5044.json', 'name': 'REP_TRUE_doc1..."
120,HIGH,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'..."
121,LOW,"{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."


In [12]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
fst_cat = ['LIE', 'FALSE', 'BARELYTRUE']
lst_cat = ['HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)
docs_each_hit = 6
gold_each_hit = 2
repetitions_each_doc = 4
actual_docs_each_hit = docs_each_hit + gold_each_hit
assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])
pool_rep = {"{}".format(categories[j]):["REP_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}
pool_dem = {"{}".format(categories[j]):["DEM_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}

def rnd_pick(sample, pool, pick, length):
    for i in range(length):
        value = random.choice(pool[pick[i]])
        sample.append(value)
        lst = pool[pick[i]]
        lst.remove(value)
        pool[pick[i]] = lst
    return sample

def single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party):
    sample1 = []
    sample2 = []
    pick_dem1 = random.sample(categories, 3)
    pick_rep2 = pick_dem1
    # Rimuovo dalle opzioni le cat dei dem
    pick_rep1 = list(set(categories) - set(pick_dem1))
    pick_dem2 = pick_rep1
    # Prendo 3 Dem
    sample1 = rnd_pick(sample1, pool_dem, pick_dem1, 3)
    # Prendo 3 Rep
    sample1 = rnd_pick(sample1, pool_rep, pick_rep1, 3)
    sample1.append('HIGH')
    sample1.append('LOW')
    sample2 = rnd_pick(sample2, pool_dem, pick_dem2, 3)
    # Prendo 3 Rep
    sample2 = rnd_pick(sample2, pool_rep, pick_rep2, 3)
    sample2.append('HIGH')
    sample2.append('LOW')
    return sample1, sample2

#sample1, sample2 = single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party)

for i in range( int((total_docs / docs_each_hit)/2) ):
    hit_sample = []
    choice1, choice2 = single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party)
    hit_sample.append(choice1)
    hit_sample.append(choice2)
    # repeat the hits j times
    for j in range(repetitions_each_doc):
        random.shuffle(hit_sample[0])
        random.shuffle(hit_sample[1])
        df.loc[len(df)] = hit_sample[0]
        df.loc[len(df)] = hit_sample[1]
        
display(df.tail())
display(filtered.tail())
display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0] * docs_each_hit) )
price = 0.5
display( 'cost:{}'.format(df.shape[0] * price) )

,d1,d2,d3,d4,d5,d6,d7,d8
75,REP_TRUE_doc5,DEM_BARELYTRUE_doc1,REP_LIE_doc4,REP_FALSE_doc2,DEM_HALFTRUE_doc9,LOW,HIGH,DEM_MOSTLYTRUE_doc6
76,DEM_LIE_doc3,DEM_TRUE_doc9,DEM_FALSE_doc1,HIGH,REP_HALFTRUE_doc4,LOW,REP_BARELYTRUE_doc7,REP_MOSTLYTRUE_doc6
77,DEM_MOSTLYTRUE_doc6,REP_TRUE_doc5,REP_FALSE_doc2,DEM_BARELYTRUE_doc1,LOW,HIGH,DEM_HALFTRUE_doc9,REP_LIE_doc4
78,DEM_TRUE_doc9,DEM_LIE_doc3,HIGH,LOW,REP_HALFTRUE_doc4,DEM_FALSE_doc1,REP_MOSTLYTRUE_doc6,REP_BARELYTRUE_doc7
79,LOW,REP_LIE_doc4,DEM_BARELYTRUE_doc1,DEM_HALFTRUE_doc9,DEM_MOSTLYTRUE_doc6,HIGH,REP_FALSE_doc2,REP_TRUE_doc5


,name,js
117,REP_TRUE_doc8,"{'id_par': '3054.json', 'name': 'REP_TRUE_doc8..."
118,REP_TRUE_doc9,"{'id_par': '4681.json', 'name': 'REP_TRUE_doc9..."
119,REP_TRUE_doc10,"{'id_par': '5044.json', 'name': 'REP_TRUE_doc1..."
120,HIGH,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'..."
121,LOW,"{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."


'HITs: 80'

'judgments:480'

'cost:40.0'

In [5]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
fst_cat = ['LIE', 'FALSE', 'BARELYTRUE']
lst_cat = ['HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)
docs_each_hit = 6
gold_each_hit = 2
repetitions_each_doc = 4
actual_docs_each_hit = docs_each_hit + gold_each_hit
assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])
pool_rep = {"{}".format(categories[j]):["REP_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}
pool_dem = {"{}".format(categories[j]):["DEM_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}

# Creo le hit deterministiche e senza golds
def create_hit_a(hit_list, total_length, fst_cat, lst_cat):
    party_per_cat = int(total_length / 2)
    for i in range(party_per_cat):
        # Non aggiungo ancora High e Low, lo faccio dopo lo Shuffle
        sample = []
        for j in fst_cat:
            value = pool_dem[j][0]
            sample.append(value)
            lst = pool_dem[j]
            lst.remove(value)
            pool_dem[j] = lst
        for j in lst_cat:
            value = pool_rep[j][0]
            sample.append(value)
            lst = pool_rep[j]
            lst.remove(value)
            pool_rep[j] = lst
        hit_list.append(sample)
    return hit_list

def create_hit_b(hit_list, total_length, fst_cat, lst_cat):
    party_per_cat = int(total_length / 2)
    for i in range(party_per_cat):
        # Non aggiungo ancora High e Low, lo faccio dopo lo Shuffle
        sample = []
        for j in fst_cat:
            value = pool_rep[j][0]
            sample.append(value)
            lst = pool_rep[j]
            lst.remove(value)
            pool_rep[j] = lst
        for j in lst_cat:
            value = pool_dem[j][0]
            sample.append(value)
            lst = pool_dem[j]
            lst.remove(value)
            pool_dem[j] = lst
        hit_list.append(sample)
    return hit_list

# Creo tutte le hits deterministiche
hits_a = [] # DEM prima | REP poi
hits_b = [] # REP prima | DEM poi
hits_a = create_hit_a(hits_a, docs_per_cat, fst_cat, lst_cat)
hits_b = create_hit_b(hits_b, docs_per_cat, fst_cat, lst_cat)

# Parto mischiandole singolarmente al loro interno
def internal_shuffle(hits):
    times = 1000
    while times >= 0:
        # Prendo uno dei 3 a caso
        index = random.sample(range(0, 3), 2)
        # Una categoria a caso
        cat = random.randrange(0, 6, 1)
        tmp = hits[index[0]][cat]
        hits[index[0]][cat] = hits[index[1]][cat]
        hits[index[1]][cat] = tmp
        times -= 1
    return hits

# Ora ho le Hits Shufflate
hits_a = internal_shuffle(hits_a)
hits_b = internal_shuffle(hits_b)

def multi_shuffle(hits_a, hits_b):
    times = 100
    while times >= 0:
        # Due nelle liste a caso | Due categorie a caso
        index = random.sample(range(0, 3), 2)
        cat_1 = random.randrange(0, 3, 1)
        cat_2 = random.randrange(3, 6, 1)
        # Scambio coppie
        fst_a = hits_a[index[0]][cat_1].split('_')
        snd_a = hits_a[index[0]][cat_2].split('_')
        fst_b = hits_b[index[1]][cat_1].split('_')
        snd_b = hits_b[index[1]][cat_2].split('_')
        if((fst_a[0] != fst_b[0]) & (snd_a[0] != snd_b[0]) & 
           (snd_a[0] == fst_b[0]) & (fst_a[0] == snd_b[0]) &
           (fst_a[0] != snd_a[0]) & (fst_b[0] != snd_b[0])):
            display("Prima")
            display(hits_a)
            display(hits_b)
            tmp1 = hits_a[index[0]][cat_1]
            hits_a[index[0]][cat_1] = hits_b[index[0]][cat_1]
            hits_b[index[1]][cat_1] = tmp1
            tmp2 = hits_a[index[0]][cat_2]
            hits_a[index[0]][cat_2] = hits_b[index[1]][cat_2]
            hits_b[index[1]][cat_2] = tmp2
            display("Dopo")
            display(hits_a)
            display(hits_b)
        times -= 1
    return hits_a, hits_b

hits_a, hits_b = multi_shuffle(hits_a, hits_b)
display(hits_a)
display(hits_b)

'Prima'

[['DEM_LIE_doc2',
  'DEM_FALSE_doc3',
  'DEM_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc1',
  'REP_MOSTLYTRUE_doc2',
  'REP_TRUE_doc1'],
 ['DEM_LIE_doc1',
  'DEM_FALSE_doc2',
  'DEM_BARELYTRUE_doc2',
  'REP_HALFTRUE_doc3',
  'REP_MOSTLYTRUE_doc3',
  'REP_TRUE_doc2'],
 ['DEM_LIE_doc3',
  'DEM_FALSE_doc1',
  'DEM_BARELYTRUE_doc3',
  'REP_HALFTRUE_doc2',
  'REP_MOSTLYTRUE_doc1',
  'REP_TRUE_doc3'],
 ['DEM_LIE_doc4',
  'DEM_FALSE_doc4',
  'DEM_BARELYTRUE_doc4',
  'REP_HALFTRUE_doc4',
  'REP_MOSTLYTRUE_doc4',
  'REP_TRUE_doc4'],
 ['DEM_LIE_doc5',
  'DEM_FALSE_doc5',
  'DEM_BARELYTRUE_doc5',
  'REP_HALFTRUE_doc5',
  'REP_MOSTLYTRUE_doc5',
  'REP_TRUE_doc5'],
 ['DEM_LIE_doc6',
  'DEM_FALSE_doc6',
  'DEM_BARELYTRUE_doc6',
  'REP_HALFTRUE_doc6',
  'REP_MOSTLYTRUE_doc6',
  'REP_TRUE_doc6'],
 ['DEM_LIE_doc7',
  'DEM_FALSE_doc7',
  'DEM_BARELYTRUE_doc7',
  'REP_HALFTRUE_doc7',
  'REP_MOSTLYTRUE_doc7',
  'REP_TRUE_doc7'],
 ['DEM_LIE_doc8',
  'DEM_FALSE_doc8',
  'DEM_BARELYTRUE_doc8',
  'REP_HALFTRUE_doc8

[['REP_LIE_doc1',
  'REP_FALSE_doc1',
  'REP_BARELYTRUE_doc3',
  'DEM_HALFTRUE_doc2',
  'DEM_MOSTLYTRUE_doc2',
  'DEM_TRUE_doc1'],
 ['REP_LIE_doc2',
  'REP_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'DEM_HALFTRUE_doc3',
  'DEM_MOSTLYTRUE_doc1',
  'DEM_TRUE_doc2'],
 ['REP_LIE_doc3',
  'REP_FALSE_doc3',
  'REP_BARELYTRUE_doc2',
  'DEM_HALFTRUE_doc1',
  'DEM_MOSTLYTRUE_doc3',
  'DEM_TRUE_doc3'],
 ['REP_LIE_doc4',
  'REP_FALSE_doc4',
  'REP_BARELYTRUE_doc4',
  'DEM_HALFTRUE_doc4',
  'DEM_MOSTLYTRUE_doc4',
  'DEM_TRUE_doc4'],
 ['REP_LIE_doc5',
  'REP_FALSE_doc5',
  'REP_BARELYTRUE_doc5',
  'DEM_HALFTRUE_doc5',
  'DEM_MOSTLYTRUE_doc5',
  'DEM_TRUE_doc5'],
 ['REP_LIE_doc6',
  'REP_FALSE_doc6',
  'REP_BARELYTRUE_doc6',
  'DEM_HALFTRUE_doc6',
  'DEM_MOSTLYTRUE_doc6',
  'DEM_TRUE_doc6'],
 ['REP_LIE_doc7',
  'REP_FALSE_doc7',
  'REP_BARELYTRUE_doc7',
  'DEM_HALFTRUE_doc7',
  'DEM_MOSTLYTRUE_doc7',
  'DEM_TRUE_doc7'],
 ['REP_LIE_doc8',
  'REP_FALSE_doc8',
  'REP_BARELYTRUE_doc8',
  'DEM_HALFTRUE_doc8

'Dopo'

[['DEM_LIE_doc2',
  'DEM_FALSE_doc3',
  'DEM_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc1',
  'REP_MOSTLYTRUE_doc2',
  'REP_TRUE_doc1'],
 ['DEM_LIE_doc1',
  'DEM_FALSE_doc2',
  'DEM_BARELYTRUE_doc2',
  'REP_HALFTRUE_doc3',
  'REP_MOSTLYTRUE_doc3',
  'REP_TRUE_doc2'],
 ['REP_LIE_doc3',
  'DEM_FALSE_doc1',
  'DEM_BARELYTRUE_doc3',
  'REP_HALFTRUE_doc2',
  'REP_MOSTLYTRUE_doc1',
  'DEM_TRUE_doc2'],
 ['DEM_LIE_doc4',
  'DEM_FALSE_doc4',
  'DEM_BARELYTRUE_doc4',
  'REP_HALFTRUE_doc4',
  'REP_MOSTLYTRUE_doc4',
  'REP_TRUE_doc4'],
 ['DEM_LIE_doc5',
  'DEM_FALSE_doc5',
  'DEM_BARELYTRUE_doc5',
  'REP_HALFTRUE_doc5',
  'REP_MOSTLYTRUE_doc5',
  'REP_TRUE_doc5'],
 ['DEM_LIE_doc6',
  'DEM_FALSE_doc6',
  'DEM_BARELYTRUE_doc6',
  'REP_HALFTRUE_doc6',
  'REP_MOSTLYTRUE_doc6',
  'REP_TRUE_doc6'],
 ['DEM_LIE_doc7',
  'DEM_FALSE_doc7',
  'DEM_BARELYTRUE_doc7',
  'REP_HALFTRUE_doc7',
  'REP_MOSTLYTRUE_doc7',
  'REP_TRUE_doc7'],
 ['DEM_LIE_doc8',
  'DEM_FALSE_doc8',
  'DEM_BARELYTRUE_doc8',
  'REP_HALFTRUE_doc8

...

'Dopo'

[['REP_LIE_doc1',
  'DEM_FALSE_doc3',
  'REP_BARELYTRUE_doc3',
  'DEM_HALFTRUE_doc3',
  'REP_MOSTLYTRUE_doc2',
  'REP_TRUE_doc1'],
 ['REP_LIE_doc3',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc2',
  'REP_MOSTLYTRUE_doc3',
  'REP_TRUE_doc2'],
 ['REP_LIE_doc1',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc1',
  'REP_MOSTLYTRUE_doc1',
  'DEM_TRUE_doc2'],
 ['DEM_LIE_doc4',
  'DEM_FALSE_doc4',
  'DEM_BARELYTRUE_doc4',
  'REP_HALFTRUE_doc4',
  'REP_MOSTLYTRUE_doc4',
  'REP_TRUE_doc4'],
 ['DEM_LIE_doc5',
  'DEM_FALSE_doc5',
  'DEM_BARELYTRUE_doc5',
  'REP_HALFTRUE_doc5',
  'REP_MOSTLYTRUE_doc5',
  'REP_TRUE_doc5'],
 ['DEM_LIE_doc6',
  'DEM_FALSE_doc6',
  'DEM_BARELYTRUE_doc6',
  'REP_HALFTRUE_doc6',
  'REP_MOSTLYTRUE_doc6',
  'REP_TRUE_doc6'],
 ['DEM_LIE_doc7',
  'DEM_FALSE_doc7',
  'DEM_BARELYTRUE_doc7',
  'REP_HALFTRUE_doc7',
  'REP_MOSTLYTRUE_doc7',
  'REP_TRUE_doc7'],
 ['DEM_LIE_doc8',
  'DEM_FALSE_doc8',
  'DEM_BARELYTRUE_doc8',
  'REP_HALFTRUE_doc8

[['REP_LIE_doc3',
  'REP_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'DEM_HALFTRUE_doc1',
  'DEM_MOSTLYTRUE_doc2',
  'DEM_TRUE_doc1'],
 ['REP_LIE_doc3',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'DEM_HALFTRUE_doc2',
  'DEM_MOSTLYTRUE_doc1',
  'REP_TRUE_doc3'],
 ['REP_LIE_doc1',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc3',
  'DEM_MOSTLYTRUE_doc3',
  'DEM_TRUE_doc3'],
 ['REP_LIE_doc4',
  'REP_FALSE_doc4',
  'REP_BARELYTRUE_doc4',
  'DEM_HALFTRUE_doc4',
  'DEM_MOSTLYTRUE_doc4',
  'DEM_TRUE_doc4'],
 ['REP_LIE_doc5',
  'REP_FALSE_doc5',
  'REP_BARELYTRUE_doc5',
  'DEM_HALFTRUE_doc5',
  'DEM_MOSTLYTRUE_doc5',
  'DEM_TRUE_doc5'],
 ['REP_LIE_doc6',
  'REP_FALSE_doc6',
  'REP_BARELYTRUE_doc6',
  'DEM_HALFTRUE_doc6',
  'DEM_MOSTLYTRUE_doc6',
  'DEM_TRUE_doc6'],
 ['REP_LIE_doc7',
  'REP_FALSE_doc7',
  'REP_BARELYTRUE_doc7',
  'DEM_HALFTRUE_doc7',
  'DEM_MOSTLYTRUE_doc7',
  'DEM_TRUE_doc7'],
 ['REP_LIE_doc8',
  'REP_FALSE_doc8',
  'REP_BARELYTRUE_doc8',
  'DEM_HALFTRUE_doc8

[['REP_LIE_doc1',
  'DEM_FALSE_doc3',
  'REP_BARELYTRUE_doc3',
  'DEM_HALFTRUE_doc3',
  'REP_MOSTLYTRUE_doc2',
  'REP_TRUE_doc1'],
 ['REP_LIE_doc3',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc2',
  'REP_MOSTLYTRUE_doc3',
  'REP_TRUE_doc2'],
 ['REP_LIE_doc1',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc1',
  'REP_MOSTLYTRUE_doc1',
  'DEM_TRUE_doc2'],
 ['DEM_LIE_doc4',
  'DEM_FALSE_doc4',
  'DEM_BARELYTRUE_doc4',
  'REP_HALFTRUE_doc4',
  'REP_MOSTLYTRUE_doc4',
  'REP_TRUE_doc4'],
 ['DEM_LIE_doc5',
  'DEM_FALSE_doc5',
  'DEM_BARELYTRUE_doc5',
  'REP_HALFTRUE_doc5',
  'REP_MOSTLYTRUE_doc5',
  'REP_TRUE_doc5'],
 ['DEM_LIE_doc6',
  'DEM_FALSE_doc6',
  'DEM_BARELYTRUE_doc6',
  'REP_HALFTRUE_doc6',
  'REP_MOSTLYTRUE_doc6',
  'REP_TRUE_doc6'],
 ['DEM_LIE_doc7',
  'DEM_FALSE_doc7',
  'DEM_BARELYTRUE_doc7',
  'REP_HALFTRUE_doc7',
  'REP_MOSTLYTRUE_doc7',
  'REP_TRUE_doc7'],
 ['DEM_LIE_doc8',
  'DEM_FALSE_doc8',
  'DEM_BARELYTRUE_doc8',
  'REP_HALFTRUE_doc8

[['REP_LIE_doc3',
  'REP_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'DEM_HALFTRUE_doc1',
  'DEM_MOSTLYTRUE_doc2',
  'DEM_TRUE_doc1'],
 ['REP_LIE_doc3',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'DEM_HALFTRUE_doc2',
  'DEM_MOSTLYTRUE_doc1',
  'REP_TRUE_doc3'],
 ['REP_LIE_doc1',
  'DEM_FALSE_doc2',
  'REP_BARELYTRUE_doc1',
  'REP_HALFTRUE_doc3',
  'DEM_MOSTLYTRUE_doc3',
  'DEM_TRUE_doc3'],
 ['REP_LIE_doc4',
  'REP_FALSE_doc4',
  'REP_BARELYTRUE_doc4',
  'DEM_HALFTRUE_doc4',
  'DEM_MOSTLYTRUE_doc4',
  'DEM_TRUE_doc4'],
 ['REP_LIE_doc5',
  'REP_FALSE_doc5',
  'REP_BARELYTRUE_doc5',
  'DEM_HALFTRUE_doc5',
  'DEM_MOSTLYTRUE_doc5',
  'DEM_TRUE_doc5'],
 ['REP_LIE_doc6',
  'REP_FALSE_doc6',
  'REP_BARELYTRUE_doc6',
  'DEM_HALFTRUE_doc6',
  'DEM_MOSTLYTRUE_doc6',
  'DEM_TRUE_doc6'],
 ['REP_LIE_doc7',
  'REP_FALSE_doc7',
  'REP_BARELYTRUE_doc7',
  'DEM_HALFTRUE_doc7',
  'DEM_MOSTLYTRUE_doc7',
  'DEM_TRUE_doc7'],
 ['REP_LIE_doc8',
  'REP_FALSE_doc8',
  'REP_BARELYTRUE_doc8',
  'DEM_HALFTRUE_doc8

In [6]:
split_11 = hits_a[indexes[0]][cat_1].split('_')
        split_12 = hits_a[indexes[1]][cat_2].split('_')
        split_21 = hits_b[indexes[0]][cat_1].split('_')
        split_22 = hits_b[indexes[1]][cat_2].split('_')
        if((split_11[0] != split_12[0]) & (split_21[0] != split_22[0]) & 
           (split_11[0] == split_22[0]) & (split_12[0] == split_21[0])):






party_per_cat = int(docs_per_cat/2) # Da qui so che docs_per_cat deve essere pari
for i in range(party_per_cat):
    # Non aggiungo ancora High e Low, lo faccio per ogni dopo lo shuffle 
    sample = []
    for j in fst_cat:
        value = pool_dem[j][0]
        sample.append(value)
        lst = pool_dem[j]
        lst.remove(value)
        pool_dem[j] = lst
    for j in lst_cat:
        value = pool_rep[j][0]
        sample.append(value)
        lst = pool_rep[j]
        lst.remove(value)
        pool_rep[j] = lst
    display(sample)
    
for i in range(party_per_cat):
    # Non aggiungo ancora High e Low, lo faccio per ogni dopo lo shuffle 
    sample = []
    for j in lst_cat:
        value = pool_dem[j][0]
        sample.append(value)
        lst = pool_dem[j]
        lst.remove(value)
        pool_dem[j] = lst
    for j in fst_cat:
        value = pool_rep[j][0]
        sample.append(value)
        lst = pool_rep[j]
        lst.remove(value)
        pool_rep[j] = lst
    display(sample)

IndentationError: unexpected indent (<ipython-input-6-f0e48c5afe56>, line 2)

In [ ]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)

docs_each_hit = 6
gold_each_hit = 2
actual_docs_each_hit = docs_each_hit + gold_each_hit

assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

repetitions_each_doc = 4

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])

doc_pool = {"{}".format(categories[j]):["{}_{}_doc{}".format(par[k], categories[j], i + 1) for k in range(len(par))for i in range(docs_per_cat_by_party)] for j in range(len(categories)) }

def sample_per_cat(pool, categories):
    sample = ['HIGH', 'LOW']
    dem_count = 0;
    rep_count = 0;
    for j in categories:
        # Cerco un candidato valido | 3 DEM e 3 REP
        found_valid = False
        while not found_valid:
            selected = random.choice(pool[j])
            # Verifico che vada bene
            temp = selected.split('_')
            if(temp[0] == 'DEM'):
                if(dem_count < 3):
                    dem_count = dem_count + 1
                    found_valid = True
            elif(rep_count < 3): # Altrimenti è REP
                rep_count = rep_count + 1
                found_valid = True
        sample.append(selected)
        # remove
        lst = doc_pool[j]
        lst.remove(selected)
        doc_pool[j] = lst
    display(sample)
    return sample

for i in range( int(total_docs / docs_each_hit) ):
    hit_sample = []
    for z in range( int(docs_each_hit / len(categories))  ):
        choice = sample_per_cat(pool = doc_pool, categories = categories)
        hit_sample.append(choice)
    # flatten list
    hit_sample = list(itertools.chain.from_iterable(hit_sample))
    # repeat the hits j times
    for j in range(repetitions_each_doc):
        random.shuffle(hit_sample)
        df.loc[len(df)] = hit_sample
    
display(doc_pool)
display(df.head())
display(filtered.head())

display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0]*docs_each_hit) )
price = 0.2
display( 'cost:{}'.format(df.shape[0]*price) )

In [ ]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)
docs_each_hit = 6
gold_each_hit = 2
actual_docs_each_hit = docs_each_hit + gold_each_hit

assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

repetitions_each_doc = 4

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])
d = df

doc_pool_rep = {"{}".format(categories[j]):["REP_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}
doc_pool_dem = {"{}".format(categories[j]):["DEM_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}

def rnd_pick(sample, pool, pick, length):
    for i in range(length):
        value = random.choice(pool[pick[i]])
        sample.append(value)
        lst = pool[pick[i]]
        lst.remove(value)
        pool[pick[i]] = lst
    return sample

def single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party):
    sample = []
    pick_dem = random.sample(categories, docs_per_cat_by_party)
    # Rimuovo dalle opzioni le cat dei dem
    pick_rep = list(set(categories) - set(pick_dem))
    # Prendo 3 Dem
    sample = rnd_pick(sample, pool_dem, pick_dem, docs_per_cat_by_party)
    # Prendo 3 Rep
    sample = rnd_pick(sample, pool_rep, pick_rep, docs_per_cat_by_party)
    sample.append('HIGH')
    sample.append('LOW')
    display(pool_rep)
    display(pool_dem)
    display(sample)
    return sample

#single_hit(doc_pool_rep, doc_pool_dem, categories, docs_per_cat_by_party)

for i in range( int(total_docs / docs_each_hit) ):
    hit_sample = []
    choice = single_hit(doc_pool_rep, doc_pool_dem, categories, docs_per_cat_by_party)
    hit_sample.append(choice)
    # flatten list
    hit_sample = list(itertools.chain.from_iterable(hit_sample))
    # repeat the hits j times
    for j in range(repetitions_each_doc):
        random.shuffle(hit_sample)
        df.loc[len(df)] = hit_sample

display(df.head())

assert False
    
display(doc_pool)
display(df.head())
display(filtered.head())
display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0] * docs_each_hit) )
price = 0.2
display( 'cost:{}'.format(df.shape[0] * price) )

In [ ]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)

docs_each_hit = 6
gold_each_hit = 2
actual_docs_each_hit = docs_each_hit + gold_each_hit

assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

repetitions_each_doc = 4

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])
d = df

doc_pool = {"{}".format(categories[j]):["{}_{}_doc{}".format(par[k], categories[j], i + 1) for k in range(len(par))for i in range(docs_per_cat_by_party)] for j in range(len(categories)) }


# Verifico la bontà del pick
def check_pick(selected, category, hit_count):
    is_ok = False
    temp = selected.split('_')
    if(temp[0] == 'DEM'):
        if(hit_count[0] > 0):
            hit_count[0] += 1
            is_ok = True
    if(temp[0] == 'REP'):
        if(hit_count[1] > 0):
            hit_count[1] += 1
            is_ok = True         
    return is_ok


def sample_per_cat(pool, categories):
    sample = ['HIGH', 'LOW']
    hit_count = [docs_per_cat_by_party, docs_per_cat_by_party]
    for j in categories:
        found_valid = False # Cerco un candidato valido | 3 DEM e 3 REP
        i = 0
        while not (found_valid & (i < docs_per_cat*2)):
            selected = random.choice(pool[j])
            found_valid = check_pick(selected, j, hit_count)
            i+=1
        if (i >= docs_per_cat*2):
            display("Eccoci dio stronzo")
        sample.append(selected)
        # Rimuovo
        lst = doc_pool[j]
        lst.remove(selected)
        doc_pool[j] = lst
    display(doc_pool)
    display(sample)
    return sample

def create_hit(pool, cat):
    for i in range( int(total_docs / docs_each_hit) ):
        hit_sample = []
        for z in range( int(docs_each_hit / len(categories))  ):
            choice = sample_per_cat(pool, cat)
            hit_sample.append(choice)
        # flatten list
        hit_sample = list(itertools.chain.from_iterable(hit_sample))
        # repeat the hits j times
        for j in range(repetitions_each_doc):
            random.shuffle(hit_sample)
            d.loc[len(d)] = hit_sample
    return d

# Creazione delle HITs
df = create_hit(doc_pool, categories)

display(doc_pool)
display(df.head())
display(filtered.head())
display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0] * docs_each_hit) )
price = 0.2
display( 'cost:{}'.format(df.shape[0] * price) )

In [ ]:

#pick1 = random.sample(categories, 3)
#display(random.choice(doc_pool_dem[pick1[0]]))
#display(random.choice(doc_pool_dem[pick1[1]]))
#display(random.choice(doc_pool_dem[pick1[2]]))

#pick2 = list(set(categories) - set(pick1))
#display(pick2)
#display(random.choice(doc_pool_rep[pick2[0]]))
#display(random.choice(doc_pool_rep[pick2[1]]))
#display(random.choice(doc_pool_rep[pick2[2]]))


categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)

docs_each_hit = 6
gold_each_hit = 2
actual_docs_each_hit = docs_each_hit + gold_each_hit

assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

repetitions_each_doc = 4

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])

doc_pool = {"{}".format(categories[j]):["{}_{}_doc{}".format(par[k], categories[j], i + 1) for k in range(len(par))for i in range(docs_per_cat_by_party)] for j in range(len(categories)) }

def sample_per_cat(pool, categories):
    sample = ['HIGH', 'LOW']
    dem_count = 0;
    rep_count = 0;
    for j in categories:
        # Cerco un candidato valido | 3 DEM e 3 REP
        found_valid = False
        while not found_valid:
            selected = random.choice(pool[j])
            # Verifico che vada bene
            temp = selected.split('_')
            if(temp[0] == 'DEM'):
                if(dem_count < 3):
                    dem_count = dem_count + 1
                    found_valid = True
            elif(rep_count < 3): # Altrimenti è REP
                rep_count = rep_count + 1
                found_valid = True
        sample.append(selected)
        # remove
        lst = doc_pool[j]
        lst.remove(selected)
        doc_pool[j] = lst
    display(doc_pool)
    return sample

for i in range( int(total_docs / docs_each_hit) ):
    hit_sample = []
    for z in range( int(docs_each_hit / len(categories))  ):
        choice = sample_per_cat(pool = doc_pool, categories = categories)
        hit_sample.append(choice)
    # flatten list
    hit_sample = list(itertools.chain.from_iterable(hit_sample))
    # repeat the hits j times
    for j in range(repetitions_each_doc):
        random.shuffle(hit_sample)
        df.loc[len(df)] = hit_sample
    
display(doc_pool)
display(df.head())
display(filtered.head())

display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0]*docs_each_hit) )
price = 0.2
display( 'cost:{}'.format(df.shape[0]*price) )